In [2]:
# Final Project Evaluation Rubrics:
# ·       2 Marks based on your understanding of data preparation.
# ·       2 Marks based on your understanding of data exploration.
# ·       4 Marks based on your understanding of data  modelling.
# ·       2 Marks based on your understanding of model evaluation.
# ·       10 Marks for successfully running the code and getting results.

In [3]:
import pandas as pd
import numpy as np
from itertools import product
from collections import Counter
from sklearn.cluster import KMeans
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
import category_encoders as ce
import warnings

pd.set_option('display.max_rows', 400)
pd.set_option('display.max_columns', 160)
pd.set_option('display.max_colwidth', 40)
warnings.filterwarnings("ignore")

In [4]:
test = pd.read_csv('test.csv')
test.head()

,ID,shop_id,item_id
0,0,5,5037
1,1,5,5320
2,2,5,5233
3,3,5,5232
4,4,5,5268


In [5]:
categories = pd.read_csv('item_categories.csv')
pd.DataFrame(categories.category_name.values.reshape(-1, 4))

,0,1,2,3
0,PC - Headsets / Headphones,Accessories - PS2,Accessories - PS3,Accessories - PS4
1,Accessories - PSP,Accessories - PSVita,Accessories - XBOX 360,Accessories - XBOX ONE
2,Tickets (Digital),Delivery of goods,Game consoles - PS2,Game consoles - PS3
3,Game consoles - PS4,Game consoles - PSP,Game consoles - PSVita,Game consoles - XBOX 360
4,Game consoles - XBOX ONE,Game consoles - Others,Games - PS2,Games - PS3
5,Games - PS4,Games - PSP,Games - PSVita,Games - XBOX 360
6,Games - XBOX ONE,Games - Game accessories,Android games - Number,MAC Games - Number
7,PC Games - Additional Editions,PC Games - Collector's Editions,PC Games - Standard Editions,PC Games - Digital
8,"Payment Cards (Cinema, Music, Games)",Payment Cards - Live!,Payment Cards - Live! (Digital),Payment cards - PSN
9,Payment cards - Windows (Digital),Cinema - Blu-Ray,Cinema - Blu-Ray 3D,Cinema - Blu-Ray 4K


In [6]:
categories['group_name'] = categories['category_name'].str.extract(r'(^[\w\s]*)')
categories['group_name'] = categories['group_name'].str.strip()
categories['group_id']  = le.fit_transform(categories.group_name.values)
categories.sample(5)

,category_name,category_id,group_name,group_id
82,Blank media (piece),82,Blank media,3
78,Programs - Educational (Digit),78,Programs,16
12,Game consoles - PS4,12,Game consoles,7
1,Accessories - PS2,1,Accessories,0
77,Programs - Educational,77,Programs,16


In [7]:
items = pd.read_csv('items.csv')

items['item_name'] = items['item_name'].str.lower()
items['item_name'] = items['item_name'].str.replace('.', '')
for i in [r'[^\w\d\s\.]', r'\bthe\b', r'\bin\b', r'\bis\b',
          r'\bfor\b', r'\bof\b', r'\bon\b', r'\band\b',  
          r'\bto\b', r'\bwith\b' , r'\byo\b']:
    items['item_name'] = items['item_name'].str.replace(i, ' ')
items['item_name'] = items['item_name'].str.replace(r'\b.\b', ' ')

items['item_name_no_space'] = items['item_name'].str.replace(' ', '')
items['item_name_first4'] = [x[:4] for x in items['item_name_no_space']]
items['item_name_first6'] = [x[:6] for x in items['item_name_no_space']]
items['item_name_first11'] = [x[:11] for x in items['item_name_no_space']]
del items['item_name_no_space']
                              
items.item_name_first4 = le.fit_transform(items.item_name_first4.values)
items.item_name_first6 = le.fit_transform(items.item_name_first6.values)
items.item_name_first11 = le.fit_transform(items.item_name_first11.values)

items = items.join(categories.set_index('category_id'), on='category_id')
items.sample(10)

,item_id,category_id,item_name,item_name_first4,item_name_first6,item_name_first11,category_name,group_name,group_id
3094,3094,58,electric light orchestra eldorado lp,1064,1957,3503,Music - Vinyl,Music,11
19098,19098,40,old new year rem,2387,4309,7787,Cinema - DVD,Cinema,5
18100,18100,57,rybnikov alexey junon avos rock op...,2853,5100,9131,Music - MP3,Music,11
1373,1373,31,air conflicts secret wars pc digi...,183,286,448,PC Games - Digital,PC Games,13
1930,1930,31,binary domain multiplayer pack pc...,492,842,1441,PC Games - Digital,PC Games,13
7236,7236,31,unmechanical pc digital version,3485,6358,11326,PC Games - Digital,PC Games,13
14587,14587,40,mizerere,2116,3867,6936,Cinema - DVD,Cinema,5
17065,17065,40,pirates caribbean curve blac...,2552,4584,8245,Cinema - DVD,Cinema,5
21604,21604,40,charodey rem,698,1241,2133,Cinema - DVD,Cinema,5
14811,14811,40,masters avengers greatest heroe...,2032,3705,6613,Cinema - DVD,Cinema,5


In [8]:
dupes = items[(items.duplicated(subset=['item_name','category_id'],keep=False))]
dupes['in_test'] = dupes.item_id.isin(test.item_id.unique())
dupes = dupes.groupby('item_name').agg({'item_id':['first','last'],'in_test':['first','last']})

dupes = dupes[(dupes[('in_test', 'first')]==False) | (dupes[('in_test', 'last')]==False)]
temp = dupes[dupes[('in_test', 'first')]==True]
keep_first = dict(zip(temp[('item_id', 'last')], temp[('item_id',  'first')]))
temp = dupes[dupes[('in_test', 'first')]==False]
keep_second = dict(zip(temp[('item_id', 'first')], temp[('item_id',  'last')]))
item_map = {**keep_first, **keep_second}

In [9]:
sales = pd.read_csv('sales_train.csv')
sales = (sales
    .query('0 < item_price < 50000 and 0 < item_cnt_day < 1001')
    .replace({
        'shop_id':{0:57, 1:58, 11:10},
        'item_id':item_map
    })    
)

sales = sales[sales['shop_id'].isin(test.shop_id.unique())]

sales['date'] = pd.to_datetime(sales.date,format='%d.%m.%Y')
sales['weekday'] = sales.date.dt.dayofweek

sales['first_sale_day'] = sales.date.dt.dayofyear 
sales['first_sale_day'] += 365 * (sales.date.dt.year-2013)
sales['first_sale_day'] = sales.groupby('item_id')['first_sale_day'].transform('min').astype('int16')

sales['revenue'] = sales['item_cnt_day']*sales['item_price']

In [10]:
temp = sales.groupby(['shop_id','weekday']).agg({'item_cnt_day':'sum'}).reset_index()
temp = pd.merge(temp, sales.groupby(['shop_id']).agg({'item_cnt_day':'sum'}).reset_index(), on='shop_id', how='left')
temp.columns = ['shop_id','weekday', 'shop_day_sales', 'shop_total_sales']
temp['day_quality'] = temp['shop_day_sales']/temp['shop_total_sales']
temp = temp[['shop_id','weekday','day_quality']]

dates = pd.DataFrame(data={'date':pd.date_range(start='2013-01-01',end='2015-11-30')})
dates['weekday'] = dates.date.dt.dayofweek
dates['month'] = dates.date.dt.month
dates['year'] = dates.date.dt.year - 2013
dates['date_block_num'] = dates['year']*12 + dates['month'] - 1
dates['first_day_of_month'] = dates.date.dt.dayofyear
dates['first_day_of_month'] += 365 * dates['year']
dates = dates.join(temp.set_index('weekday'), on='weekday')
dates = dates.groupby(['date_block_num','shop_id','month','year']).agg({'day_quality':'sum','first_day_of_month':'min'}).reset_index()

dates.query('shop_id == 28').head(15)

,date_block_num,shop_id,month,year,day_quality,first_day_of_month
18,0,28,1,0,4.300037,1
60,1,28,2,0,4.000000,32
102,2,28,3,0,4.605952,60
144,3,28,4,0,4.193211,91
186,4,28,5,0,4.333970,121
228,5,28,6,0,4.472819,152
270,6,28,7,0,4.287644,182
312,7,28,8,0,4.492718,213
354,8,28,9,0,4.313648,244
396,9,28,10,0,4.300037,274


In [11]:
sales = (sales
     .groupby(['date_block_num', 'shop_id', 'item_id'])
     .agg({
         'item_cnt_day':'sum', 
         'revenue':'sum',
         'first_sale_day':'first'
     })
     .reset_index()
     .rename(columns={'item_cnt_day':'item_cnt'})
)
sales.sample(5)

,date_block_num,shop_id,item_id,item_cnt,revenue,first_sale_day
579666,12,52,17818,2.0,198.67,199
958520,23,6,8852,3.0,1197.00,631
971679,23,25,5090,1.0,449.00,54
958818,23,6,13405,7.0,105715.00,634
827720,19,26,13025,1.0,199.00,535


In [12]:
df = [] 
for block_num in sales['date_block_num'].unique():
    cur_shops = sales.loc[sales['date_block_num'] == block_num, 'shop_id'].unique()
    cur_items = sales.loc[sales['date_block_num'] == block_num, 'item_id'].unique()
    df.append(np.array(list(product(*[cur_shops, cur_items, [block_num]]))))

df = pd.DataFrame(np.vstack(df), columns=['shop_id', 'item_id', 'date_block_num'])
df.head()

,shop_id,item_id,date_block_num
0,2,27,0
1,2,33,0
2,2,317,0
3,2,438,0
4,2,471,0


In [13]:
test['date_block_num'] = 34
del test['ID']

In [14]:
df = pd.concat([df,test]).fillna(0)
df = df.reset_index()
del df['index']

In [15]:
df = pd.merge(df, sales, on=['shop_id', 'item_id', 'date_block_num'], how='left').fillna(0)
df = pd.merge(df, dates, on=['date_block_num','shop_id'], how='left')
df = pd.merge(df, items.drop(columns=['item_name','group_name','category_name']), on='item_id', how='left')

In [16]:
shops = pd.read_csv('shops.csv')

shops_cats = pd.DataFrame(
    np.array(list(product(*[df['shop_id'].unique(), df['category_id'].unique()]))),
    columns =['shop_id', 'category_id']
)
temp = df.groupby(['category_id', 'shop_id']).agg({'item_cnt':'sum'}).reset_index()
temp2 = temp.groupby('shop_id').agg({'item_cnt':'sum'}).rename(columns={'item_cnt':'shop_total'})
temp = temp.join(temp2, on='shop_id')
temp['category_proportion'] = temp['item_cnt']/temp['shop_total']
temp = temp[['shop_id', 'category_id', 'category_proportion']]
shops_cats = pd.merge(shops_cats, temp, on=['shop_id','category_id'], how='left')
shops_cats = shops_cats.fillna(0)

shops_cats = shops_cats.pivot(index='shop_id', columns=['category_id'])
kmeans = KMeans(n_clusters=7, random_state=0).fit(shops_cats)
shops_cats['shop_cluster'] = kmeans.labels_.astype('int8')

shops = shops.join(shops_cats['shop_cluster'], on='shop_id')

In [17]:
shops.dropna(inplace=True)

shops['shop_name'] = shops['shop_name'].str.lower()
shops['shop_name'] = shops['shop_name'].str.replace(r'[^\w\d\s]', ' ')

shops['shop_type'] = 'regular'

shops.loc[shops['shop_name'].str.contains(r'tc'), 'shop_type'] = 'tc'
shops.loc[shops['shop_name'].str.contains(r'mall|center|mega'), 'shop_type'] = 'mall'
shops.loc[shops['shop_id'].isin([9,20]), 'shop_type'] = 'special'
shops.loc[shops['shop_id'].isin([12,55]), 'shop_type'] = 'online'

shops['shop_city'] = shops['shop_name'].str.split().str[0]
shops.loc[shops['shop_id'].isin([12,55]), 'shop_city'] = 'online'
shops.shop_city = le.fit_transform(shops.shop_city.values)
shops.shop_type = le.fit_transform(shops.shop_type.values)
shops.head()

,shop_name,shop_id,shop_cluster,shop_type,shop_city
2,adygea tc mega,2,0.0,0,0
3,balashikha tc oktyabr kinomir,3,3.0,3,1
4,volga tc volga mall,4,3.0,0,22
5,vologda sec marmelad,5,3.0,2,23
6,voronezh plekhanovskaya 13,6,3.0,2,24


In [18]:
df = pd.merge(df, shops.drop(columns='shop_name'), on='shop_id', how='left')
df.head()

,shop_id,item_id,date_block_num,item_cnt,revenue,first_sale_day,month,year,day_quality,first_day_of_month,category_id,item_name_first4,item_name_first6,item_name_first11,group_id,shop_cluster,shop_type,shop_city
0,2,27,0,1.0,2499.0,2.0,1,0,4.381749,1,19,2,2,2,8,0.0,0,0
1,2,33,0,1.0,499.0,2.0,1,0,4.381749,1,37,415,720,1203,5,0.0,0,0
2,2,317,0,1.0,299.0,4.0,1,0,4.381749,1,45,31,38,52,4,0.0,0,0
3,2,438,0,1.0,299.0,19.0,1,0,4.381749,1,45,31,38,57,4,0.0,0,0
4,2,471,0,2.0,798.0,3.0,1,0,4.381749,1,49,30,37,51,4,0.0,0,0


In [19]:
df['first_sale_day'] = df.groupby('item_id')['first_sale_day'].transform('max').astype('int16')
df.loc[df['first_sale_day']==0, 'first_sale_day'] = 1035
df['prev_days_on_sale'] = [max(idx) for idx in zip(df['first_day_of_month']-df['first_sale_day'],[0]*len(df))]
del df['first_day_of_month']

In [20]:
del sales, categories, shops, shops_cats, temp, temp2, test, dupes, item_map, 
df.head()

,shop_id,item_id,date_block_num,item_cnt,revenue,first_sale_day,month,year,day_quality,category_id,item_name_first4,item_name_first6,item_name_first11,group_id,shop_cluster,shop_type,shop_city,prev_days_on_sale
0,2,27,0,1.0,2499.0,2,1,0,4.381749,19,2,2,2,8,0.0,0,0,0
1,2,33,0,1.0,499.0,2,1,0,4.381749,37,415,720,1203,5,0.0,0,0,0
2,2,317,0,1.0,299.0,4,1,0,4.381749,45,31,38,52,4,0.0,0,0,0
3,2,438,0,1.0,299.0,19,1,0,4.381749,45,31,38,57,4,0.0,0,0,0
4,2,471,0,2.0,798.0,3,1,0,4.381749,49,30,37,51,4,0.0,0,0,0


In [21]:
df['item_cnt_unclipped'] = df['item_cnt']
df['item_cnt'] = df['item_cnt'].clip(0, 20)

In [22]:
def downcast(df):
    float_cols = [c for c in df if df[c].dtype in ["float64"]]
    int_cols = [c for c in df if df[c].dtype in ['int64']]
    df[float_cols] = df[float_cols].astype('float32')
    df[int_cols] = df[int_cols].astype('int16')
    return df
df = downcast(df)

In [23]:
df['item_age'] = (df['date_block_num'] - df.groupby('item_id')['date_block_num'].transform('min')).astype('int8')
df['item_name_first4_age'] = (df['date_block_num'] - df.groupby('item_name_first4')['date_block_num'].transform('min')).astype('int8')
df['item_name_first6_age'] = (df['date_block_num'] - df.groupby('item_name_first6')['date_block_num'].transform('min')).astype('int8')
df['item_name_first11_age'] = (df['date_block_num'] - df.groupby('item_name_first11')['date_block_num'].transform('min')).astype('int8')
df['category_age'] = (df['date_block_num'] - df.groupby('category_id')['date_block_num'].transform('min')).astype('int8')
df['group_age'] = (df['date_block_num'] - df.groupby('group_id')['date_block_num'].transform('min')).astype('int8')
df['shop_age'] = (df['date_block_num'] - df.groupby('shop_id')['date_block_num'].transform('min')).astype('int8')

In [24]:
temp = df.query('item_cnt > 0').groupby(['item_id','shop_id']).agg({'date_block_num':'min'}).reset_index()
temp.columns = ['item_id', 'shop_id', 'item_shop_first_sale']
df = pd.merge(df, temp, on=['item_id','shop_id'], how='left')
df['item_shop_first_sale'] = df['item_shop_first_sale'].fillna(50)
df['item_age_if_shop_sale'] = (df['date_block_num'] > df['item_shop_first_sale']) * df['item_age']
df['item_age_without_shop_sale'] = (df['date_block_num'] <= df['item_shop_first_sale']) * df['item_age']
del df['item_shop_first_sale']

In [25]:
def agg_cnt_col(df, merging_cols, new_col,aggregation):
    temp = df.groupby(merging_cols).agg(aggregation).reset_index()
    temp.columns = merging_cols + [new_col]
    df = pd.merge(df, temp, on=merging_cols, how='left')
    return df

df = agg_cnt_col(df, ['date_block_num','item_id'],'item_cnt_all_shops',{'item_cnt':'mean'})
df = agg_cnt_col(df, ['date_block_num','category_id','shop_id'],'item_cnt_all_shops_median',{'item_cnt':'median'}) 
df = agg_cnt_col(df, ['date_block_num','category_id','shop_id'],'category_cnt',{'item_cnt':'mean'})
df = agg_cnt_col(df, ['date_block_num','category_id','shop_id'],'category_cnt_median',{'item_cnt':'median'}) 
df = agg_cnt_col(df, ['date_block_num','category_id'],'category_cnt_all_shops',{'item_cnt':'mean'})
df = agg_cnt_col(df, ['date_block_num','category_id'],'category_cnt_all_shops_median',{'item_cnt':'median'})
df = agg_cnt_col(df, ['date_block_num','group_id','shop_id'],'group_cnt',{'item_cnt':'mean'})
df = agg_cnt_col(df, ['date_block_num','group_id'],'group_cnt_all_shops',{'item_cnt':'mean'})
df = agg_cnt_col(df, ['date_block_num','shop_id'],'shop_cnt',{'item_cnt':'mean'})
df = agg_cnt_col(df, ['date_block_num','shop_city'],'city_cnt',{'item_cnt':'mean'})

In [26]:
def new_item_sales(df, merging_cols, new_col):
    temp = (
        df
        .query('item_age==0')
        .groupby(merging_cols)['item_cnt']
        .mean()
        .reset_index()
        .rename(columns={'item_cnt': new_col})
    )
    df = pd.merge(df, temp, on=merging_cols, how='left')
    return df

df = new_item_sales(df, ['date_block_num','category_id','shop_id'], 'new_items_in_cat')
df = new_item_sales(df, ['date_block_num','category_id'], 'new_items_in_cat_all_shops')

In [27]:
def agg_price_col(df, merging_cols, new_col):
    temp = df.groupby(merging_cols).agg({'revenue':'sum','item_cnt_unclipped':'sum'}).reset_index()
    temp[new_col] = temp['revenue']/temp['item_cnt_unclipped']
    temp = temp[merging_cols + [new_col]]
    df = pd.merge(df, temp, on=merging_cols, how='left')
    return df

df = agg_price_col(df,['date_block_num','item_id'],'item_price')
df = agg_price_col(df,['date_block_num','category_id'],'category_price')
df = agg_price_col(df,['date_block_num'],'block_price')

In [28]:
df = downcast(df)

In [29]:
def lag_feature(df, lag, col, merge_cols):        
    temp = df[merge_cols + [col]]
    temp = temp.groupby(merge_cols).agg({f'{col}':'first'}).reset_index()
    temp.columns = merge_cols + [f'{col}_lag{lag}']
    temp['date_block_num'] += lag
    df = pd.merge(df, temp, on=merge_cols, how='left')
    df[f'{col}_lag{lag}'] = df[f'{col}_lag{lag}'].fillna(0).astype('float32')
    return df

In [ ]:
lag12_cols = {
    'item_cnt':['date_block_num', 'shop_id', 'item_id'],
    'item_cnt_all_shops':['date_block_num', 'item_id'],
    'category_cnt':['date_block_num', 'shop_id', 'category_id'],
    'category_cnt_all_shops':['date_block_num', 'category_id'],
    'group_cnt':['date_block_num', 'shop_id', 'group_id'],
    'group_cnt_all_shops':['date_block_num', 'group_id'],
    'shop_cnt':['date_block_num', 'shop_id'],
    'city_cnt':['date_block_num', 'shop_city'],
    'new_items_in_cat':['date_block_num', 'shop_id', 'category_id'],
    'new_items_in_cat_all_shops':['date_block_num', 'category_id']
}
for col,merge_cols in lag12_cols.items():
    df[f'{col}_lag1to12'] = 0
    for i in range(1,13):
        df = lag_feature(df, i, col, merge_cols)
        df[f'{col}_lag1to12'] += df[f'{col}_lag{i}']
        if i > 2:
            del df[f'{col}_lag{i}']
    if col == 'item_cnt':
        del df[f'{col}_lag1']
        del df[f'{col}_lag2']        
    else:
        del df[col]

In [ ]:
lag2_cols = {
    'item_cnt_unclipped':['date_block_num', 'shop_id', 'item_id'],
    'item_cnt_all_shops_median':['date_block_num', 'item_id'],
    'category_cnt_median':['date_block_num', 'shop_id', 'category_id'],
    'category_cnt_all_shops_median':['date_block_num', 'category_id']
}
for col in lag2_cols:
    df = lag_feature(df, 1, col, merge_cols)
    df = lag_feature(df, 2, col, merge_cols)
    if col!='item_cnt_unclipped':
        del df[col]

In [ ]:
df['item_cnt_diff'] = df['item_cnt_unclipped_lag1']/df['item_cnt_lag1to12']
df['item_cnt_all_shops_diff'] = df['item_cnt_all_shops_lag1']/df['item_cnt_all_shops_lag1to12']
df['category_cnt_diff'] = df['category_cnt_lag1']/df['category_cnt_lag1to12']
df['category_cnt_all_shops_diff'] = df['category_cnt_all_shops_lag1']/df['category_cnt_all_shops_lag1to12']

In [ ]:
df = lag_feature(df, 1, 'category_price',['date_block_num', 'category_id'])
df = lag_feature(df, 1, 'block_price',['date_block_num'])
del df['category_price'], df['block_price']

In [ ]:
df.loc[(df['item_age']>0) & (df['item_cnt_lag1to12'].isna()), 'item_cnt_lag1to12'] = 0
df.loc[(df['category_age']>0) & (df['category_cnt_lag1to12'].isna()), 'category_cnt_lag1to12'] = 0
df.loc[(df['group_age']>0) & (df['group_cnt_lag1to12'].isna()), 'group_cnt_lag1to12'] = 0

In [ ]:
df['item_cnt_lag1to12'] /= [min(idx) for idx in zip(df['item_age'],df['shop_age'],[12]*len(df))]
df['item_cnt_all_shops_lag1to12'] /= [min(idx) for idx in zip(df['item_age'],[12]*len(df))]
df['category_cnt_lag1to12'] /= [min(idx) for idx in zip(df['category_age'],df['shop_age'],[12]*len(df))]
df['category_cnt_all_shops_lag1to12'] /= [min(idx) for idx in zip(df['category_age'],[12]*len(df))]
df['group_cnt_lag1to12'] /= [min(idx) for idx in zip(df['group_age'],df['shop_age'],[12]*len(df))]
df['group_cnt_all_shops_lag1to12'] /= [min(idx) for idx in zip(df['group_age'],[12]*len(df))]
df['city_cnt_lag1to12'] /= [min(idx) for idx in zip(df['date_block_num'],[12]*len(df))]
df['shop_cnt_lag1to12'] /= [min(idx) for idx in zip(df['shop_age'],[12]*len(df))]
df['new_items_in_cat_lag1to12'] /= [min(idx) for idx in zip(df['category_age'],df['shop_age'],[12]*len(df))]
df['new_items_in_cat_all_shops_lag1to12'] /= [min(idx) for idx in zip(df['category_age'],[12]*len(df))]

In [ ]:
df = downcast(df)

In [ ]:
def past_information(df, merging_cols, new_col, aggregation):
    temp = []
    for i in range(1,35):
        block = df.query(f'date_block_num < {i}').groupby(merging_cols).agg(aggregation).reset_index()
        block.columns = merging_cols + [new_col]
        block['date_block_num'] = i
        block = block[block[new_col]>0]
        temp.append(block)
    temp = pd.concat(temp)
    df = pd.merge(df, temp, on=['date_block_num']+merging_cols, how='left')
    return df

df = past_information(df, ['item_id'],'last_item_price',{'item_price':'last'})
df = past_information(df, ['shop_id','item_id'],'item_cnt_sum_alltime',{'item_cnt':'sum'})
df = past_information(df, ['item_id'],'item_cnt_sum_alltime_allshops',{'item_cnt':'sum'})

del df['revenue'], df['item_cnt_unclipped'], df['item_price']

In [ ]:
df['relative_price_item_block_lag1'] = df['last_item_price']/df['block_price_lag1']

In [ ]:
df['item_cnt_per_day_alltime'] = (df['item_cnt_sum_alltime']/df['prev_days_on_sale']).fillna(0)
df['item_cnt_per_day_alltime_allshops'] = (df['item_cnt_sum_alltime_allshops']/df['prev_days_on_sale']).fillna(0)

In [ ]:
import gc
gc.collect()
df = downcast(df)

In [ ]:
def matching_name_cat_age(df,n,all_shops):
    temp_cols = [f'same_name{n}catage_cnt','date_block_num', f'item_name_first{n}','item_age','category_id']
    if all_shops:
        temp_cols[0] += '_all_shops'
    else:
        temp_cols += ['shop_id']
    temp = []
    for i in range(1,35):
        block = (
            df
            .query(f'date_block_num < {i}')
            .groupby(temp_cols[2:])
            .agg({'item_cnt':'mean'})
            .reset_index()
            .rename(columns={'item_cnt':temp_cols[0]})
        )
        block = block[block[temp_cols[0]]>0]
        block['date_block_num'] = i
        temp.append(block)
    temp = pd.concat(temp)
    df = pd.merge(df, temp, on=temp_cols[1:], how='left')
    return df

for n in [4,6,11]:
    for all_shops in [True,False]:
        df = matching_name_cat_age(df,n,all_shops)

In [ ]:
df = downcast(df)
int8_cols = [
    'item_cnt','month','group_id','shop_type',
    'shop_city','shop_id','date_block_num','category_id',
    'item_age',
]
int16_cols = [
    'item_id','item_name_first4',
    'item_name_first6','item_name_first11'
]
for col in int8_cols:
    df[col] = df[col].astype('int8')
for col in int16_cols:
    df[col] = df[col].astype('int16')

In [ ]:
def nearby_item_data(df,col):
    if col in ['item_cnt_unclipped_lag1','item_cnt_lag1to12']:
        cols = ['date_block_num', 'shop_id', 'item_id']
        temp = df[cols + [col]] 
    else:
        cols = ['date_block_num', 'item_id']
        temp = df.groupby(cols).agg({col:'first'}).reset_index()[cols + [col]]   
    
    temp.columns = cols + [f'below_{col}']
    temp['item_id'] += 1
    df = pd.merge(df, temp, on=cols, how='left')
    
    temp.columns = cols + [f'above_{col}']
    temp['item_id'] -= 2
    df = pd.merge(df, temp, on=cols, how='left')
    
    return df

item_cols = ['item_cnt_unclipped_lag1','item_cnt_lag1to12',
             'item_cnt_all_shops_lag1','item_cnt_all_shops_lag1to12']
for col in item_cols:
    df = nearby_item_data(df,col)
    
del temp

In [ ]:
results = Counter()
items['item_name'].str.split().apply(results.update)

words = []
cnts = []
for key, value in results.items():
    words.append(key)
    cnts.append(value)
    
counts = pd.DataFrame({'word':words,'count':cnts})
common_words = counts.query('count>200').word.to_list()
for word in common_words:
    items[f'{word}_in_name'] = items['item_name'].str.contains(word).astype('int8')
drop_cols = [
    'item_id','category_id','item_name','item_name_first4',
    'item_name_first6','item_name_first11',
    'category_name','group_name','group_id'
]
items = items.drop(columns=drop_cols)

In [ ]:
df = df.join(items, on='item_id')

In [ ]:
def binary_encode(df, letters, cols):
    encoder = ce.BinaryEncoder(cols=[f'item_name_first{letters}'], return_df=True)
    temp = encoder.fit_transform(df[f'item_name_first{letters}'])
    df = pd.concat([df,temp], axis=1)
    del df[f'item_name_first{letters}_0']
    name_cols = [f'item_name_first{letters}_{x}' for x in range(1,cols)]
    df[name_cols] = df[name_cols].astype('int8')
    return df

df = binary_encode(df, 11, 15)
    
del df['item_name_first4'], df['item_name_first6']

In [ ]:
df.to_pickle('df_complete.pkl')

In [ ]:
%reset -f

In [ ]:
import pandas as pd
import numpy as np
import lightgbm as lgb
import shap
from plotly.subplots import make_subplots
import plotly.graph_objects as go

pd.set_option('display.max_rows', 160)
pd.set_option('display.max_columns', 160)
pd.set_option('display.max_colwidth', 30)

import warnings
warnings.filterwarnings("ignore")

In [ ]:
df = pd.read_pickle('df_complete.pkl')

X_train = df[~df.date_block_num.isin([0,1,33,34])]
y_train = X_train['item_cnt']
del X_train['item_cnt']

X_val = df[df['date_block_num']==33]
y_val = X_val['item_cnt']
del X_val['item_cnt']

X_test = df[df['date_block_num']==34].drop(columns='item_cnt')
X_test = X_test.reset_index()
del X_test['index']

del df

In [ ]:
def build_lgb_model(params, X_train, X_val, y_train, y_val, cat_features):
    lgb_train = lgb.Dataset(X_train, y_train)
    lgb_val = lgb.Dataset(X_val, y_val)
    model = lgb.train(params=params, train_set=lgb_train, valid_sets=(lgb_train, lgb_val), verbose_eval=50,
                     categorical_feature=cat_features)
    return model

In [ ]:
params = {
    'objective': 'rmse',
    'metric': 'rmse',
    'num_leaves': 1500,
    'min_data_in_leaf':1,
    'feature_fraction':0.7,
    'learning_rate': 0.2,
    'num_rounds': 1000,
    'early_stopping_rounds': 300,
    'seed': 1
}
cat_features = ['category_id','month','shop_id','shop_city']

lgb_model = build_lgb_model(params, X_train, X_val, y_train, y_val, cat_features)

lgb_model.save_model('initial_lgb_model.txt')

In [ ]:
# #lgb_model = lgb.Booster(model_file='../input/files-top-scoring-notebook-output-exploration/initial_lgb_model.txt')#
# #lgb_model.params['objective'] = 'rmse'
# [LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 1.416154 seconds.
# You can set `force_row_wise=true` to remove the overhead.
# And if memory is not enough, you can set `force_col_wise=true`.
# [LightGBM] [Info] Total Bins 14908
# [LightGBM] [Info] Number of data points in the train set: 7795360, number of used features: 155
# [LightGBM] [Info] Start training from score 0.312795
# Training until validation scores don't improve for 300 rounds
# [50]	training's rmse: 1.21005	valid_1's rmse: 1.05752
# [100]	training's rmse: 1.17596	valid_1's rmse: 1.03321
# [150]	training's rmse: 1.14392	valid_1's rmse: 1.01033
# [200]	training's rmse: 1.11384	valid_1's rmse: 0.989309
# [250]	training's rmse: 1.08544	valid_1's rmse: 0.969668
# [300]	training's rmse: 1.0589	valid_1's rmse: 0.951604
# [350]	training's rmse: 1.03395	valid_1's rmse: 0.935153
# [400]	training's rmse: 1.01053	valid_1's rmse: 0.919638
# [450]	training's rmse: 0.988612	valid_1's rmse: 0.905531
# [500]	training's rmse: 0.967996	valid_1's rmse: 0.892496
# [550]	training's rmse: 0.948607	valid_1's rmse: 0.880391
# [600]	training's rmse: 0.93042	valid_1's rmse: 0.869254
# [650]	training's rmse: 0.91339	valid_1's rmse: 0.858901
# [700]	training's rmse: 0.897367	valid_1's rmse: 0.849566
# [750]	training's rmse: 0.882375	valid_1's rmse: 0.840881
# [800]	training's rmse: 0.868255	valid_1's rmse: 0.832656
# [850]	training's rmse: 0.855097	valid_1's rmse: 0.825412
# [900]	training's rmse: 0.842705	valid_1's rmse: 0.818657
# [950]	training's rmse: 0.831078	valid_1's rmse: 0.812427
# [1000]	training's rmse: 0.820193	valid_1's rmse: 0.80669
# [1050]	training's rmse: 0.809977	valid_1's rmse: 0.8013
# [1100]	training's rmse: 0.800331	valid_1's rmse: 0.79649
# [1150]	training's rmse: 0.791313	valid_1's rmse: 0.792174
# [1200]	training's rmse: 0.782781	valid_1's rmse: 0.788144
# [1250]	training's rmse: 0.774754	valid_1's rmse: 0.784316
# [1300]	training's rmse: 0.767152	valid_1's rmse: 0.780932
# [1350]	training's rmse: 0.759998	valid_1's rmse: 0.777849
# [1400]	training's rmse: 0.753232	valid_1's rmse: 0.775004
# [1450]	training's rmse: 0.746851	valid_1's rmse: 0.772358
# [1500]	training's rmse: 0.740852	valid_1's rmse: 0.769921
# [1550]	training's rmse: 0.735161	valid_1's rmse: 0.767781
# [1600]	training's rmse: 0.729812	valid_1's rmse: 0.765695
# [1650]	training's rmse: 0.724687	valid_1's rmse: 0.763856
# [1700]	training's rmse: 0.719816	valid_1's rmse: 0.762079
# [1750]	training's rmse: 0.7152	valid_1's rmse: 0.760419
# [1800]	training's rmse: 0.710812	valid_1's rmse: 0.758947
# [1850]	training's rmse: 0.706603	valid_1's rmse: 0.757587
# [1900]	training's rmse: 0.702645	valid_1's rmse: 0.75633
# [1950]	training's rmse: 0.698835	valid_1's rmse: 0.755235
# [2000]	training's rmse: 0.695194	valid_1's rmse: 0.754221
# [2050]	training's rmse: 0.691711	valid_1's rmse: 0.753319
# [2100]	training's rmse: 0.688318	valid_1's rmse: 0.752375
# [2150]	training's rmse: 0.685108	valid_1's rmse: 0.751599
# [2200]	training's rmse: 0.682036	valid_1's rmse: 0.750801
# [2250]	training's rmse: 0.67905	valid_1's rmse: 0.750157
# [2300]	training's rmse: 0.676175	valid_1's rmse: 0.749571
# [2350]	training's rmse: 0.673447	valid_1's rmse: 0.748911
# [2400]	training's rmse: 0.67081	valid_1's rmse: 0.748393
# [2450]	training's rmse: 0.66822	valid_1's rmse: 0.747931
# [2500]	training's rmse: 0.665713	valid_1's rmse: 0.747541
# [2550]	training's rmse: 0.663291	valid_1's rmse: 0.747132
# [2600]	training's rmse: 0.660944	valid_1's rmse: 0.746756
# [2650]	training's rmse: 0.658674	valid_1's rmse: 0.746392
# [2700]	training's rmse: 0.656481	valid_1's rmse: 0.746022
# [2750]	training's rmse: 0.654366	valid_1's rmse: 0.745711
# [2800]	training's rmse: 0.652264	valid_1's rmse: 0.745388
# [2850]	training's rmse: 0.650207	valid_1's rmse: 0.745065
# [2900]	training's rmse: 0.648253	valid_1's rmse: 0.744751
# [2950]	training's rmse: 0.646323	valid_1's rmse: 0.744475
# [3000]	training's rmse: 0.644453	valid_1's rmse: 0.744193
# [3050]	training's rmse: 0.642599	valid_1's rmse: 0.743902
# [3100]	training's rmse: 0.640804	valid_1's rmse: 0.743725
# [3150]	training's rmse: 0.639057	valid_1's rmse: 0.743539
# [3200]	training's rmse: 0.637307	valid_1's rmse: 0.743374
# [3250]	training's rmse: 0.635608	valid_1's rmse: 0.743134
# [3300]	training's rmse: 0.633968	valid_1's rmse: 0.742984
# [3350]	training's rmse: 0.63238	valid_1's rmse: 0.742871
# [3400]	training's rmse: 0.63081	valid_1's rmse: 0.742748
# [3450]	training's rmse: 0.629256	valid_1's rmse: 0.742652
# [3500]	training's rmse: 0.627738	valid_1's rmse: 0.742513
# [3550]	training's rmse: 0.626297	valid_1's rmse: 0.742407
# [3600]	training's rmse: 0.624844	valid_1's rmse: 0.742286
# [3650]	training's rmse: 0.623437	valid_1's rmse: 0.742172
# [3700]	training's rmse: 0.62206	valid_1's rmse: 0.742032
# [3750]	training's rmse: 0.620678	valid_1's rmse: 0.74194
# [3800]	training's rmse: 0.619301	valid_1's rmse: 0.741853
# [3850]	training's rmse: 0.617943	valid_1's rmse: 0.741781
# [3900]	training's rmse: 0.616647	valid_1's rmse: 0.741781
# [3950]	training's rmse: 0.615351	valid_1's rmse: 0.741694
# [4000]	training's rmse: 0.614055	valid_1's rmse: 0.741648
# [4050]	training's rmse: 0.612811	valid_1's rmse: 0.741601
# [4100]	training's rmse: 0.611566	valid_1's rmse: 0.741532
# [4150]	training's rmse: 0.610341	valid_1's rmse: 0.741495
# [4200]	training's rmse: 0.609122	valid_1's rmse: 0.741473
# [4250]	training's rmse: 0.607937	valid_1's rmse: 0.741471
# [4300]	training's rmse: 0.606763	valid_1's rmse: 0.741463
# [4350]	training's rmse: 0.605623	valid_1's rmse: 0.741426
# [4400]	training's rmse: 0.604459	valid_1's rmse: 0.741439
# [4450]	training's rmse: 0.603363	valid_1's rmse: 0.741423
# [4500]	training's rmse: 0.602261	valid_1's rmse: 0.741461
# [4550]	training's rmse: 0.60115	valid_1's rmse: 0.741455
# [4600]	training's rmse: 0.600077	valid_1's rmse: 0.741345
# [4650]	training's rmse: 0.599039	valid_1's rmse: 0.741285
# [4700]	training's rmse: 0.597984	valid_1's rmse: 0.741252
# [4750]	training's rmse: 0.596931	valid_1's rmse: 0.741241
# [4800]	training's rmse: 0.595917	valid_1's rmse: 0.741211
# [4850]	training's rmse: 0.59492	valid_1's rmse: 0.741204
# [4900]	training's rmse: 0.593936	valid_1's rmse: 0.741155
# [4950]	training's rmse: 0.59294	valid_1's rmse: 0.741199
# [5000]	training's rmse: 0.591969	valid_1's rmse: 0.741209
# Did not meet early stopping. Best iteration is:
# [5000]	training's rmse: 0.591969	valid_1's rmse: 0.741209
# <lightgbm.basic.Booster at 0x1bae2324700>

In [ ]:
submission = pd.read_csv('sample_submission.csv')
submission['item_cnt_month'] = lgb_model.predict(X_test).clip(0,20)
submission[['ID', 'item_cnt_month']].to_csv('initial_lgb_submission.csv', index=False)